In [1]:
import os
os.chdir('/home/evanlee/PBMC_Hao/')


In [5]:
# import AD2_w_utils as ad2
from AD2_w_utils import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import scipy
import scanpy as sc
import sklearn
import copy
# %matplotlib inline


In [7]:
# Demo
url_1 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/ASV_vst.txt'
url_2 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/ASV_table.txt'
url_3 = 'https://raw.githubusercontent.com/YinchengChen23/ADlasso/main/data/crc_zeller/metadata.txt'
Data = pd.read_csv(url_1, sep="\t")
Data = Data.T           # Variance-stabilizing transformation was conducted by DESeq2
# we using z-normalization data as input-data
Data_std = scipy.stats.zscore(Data, axis=0, ddof=0)
RawData = pd.read_csv(url_2, sep="\t")
RawData = RawData.T  # Raw count data, was used as an assessment of prevalence
Cohort = pd.read_csv(url_3, sep="\t")                        # Metadata
Label = Cohort['Class'].tolist()

print('This data contains',
      Data_std.shape[0], 'samples and', Data_std.shape[1], 'features')
print(Label[0:10], np.unique(Label))

# get_prevalence(data, [0,1,2, ..., 129])
pvl0 = get_prevalence(RawData, np.arange(RawData.shape[0]))
res0 = ADlasso2(lmbd=1e-5, alpha=0.9, echo=True)
start = time.time()
res0.fit(Data_std, Label, pvl0)  # .fit(X, y, prevalence)
# minimum epoch =  9999 ; minimum lost =  6.27363842795603e-05 ; diff weight =  0.002454951871186495
end = time.time()

print('median of selected prevalence :', np.median(
    [pvl0[i] for i, w in enumerate(res0.feature_set) if w != 0]))
print('total selected feature :', np.sum(res0.feature_set))
print("Total cost：%f sec" % (end - start))

# Export selection result
res0.writeList('./demo_selectedList.txt', Data_std.columns)

This data contains 129 samples and 6661 features
['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal'] ['Cancer' 'Normal']
median of selected prevalence : 0.29457364341085274
total selected feature : 483
Total cost：0.384000 sec


## Read PBMC dataset

In [8]:
# data_path = '/Users/evanli/Documents/Research_datasets/PBMC_Hao/'
os.chdir('/home/evanlee/PBMC_Hao/')
data_path = ''
adata_raw = sc.read(data_path + 'Hao_PBMC.h5ad')
print(adata_raw.shape)  # row is cells, column is gene
# (161764, 20568)


(161764, 20568)


In [9]:
# Metadata
types_l1 = adata_raw.obs['celltype.l1'].unique()  # 8
types_l2 = adata_raw.obs['celltype.l2'].unique()  # 31
types_l3 = adata_raw.obs['celltype.l3'].unique()  # 58

celltype_df = adata_raw.obs[['celltype.l1', 'celltype.l2', 'cell_type']]
celltype_df = celltype_df.sort_values(['celltype.l1', 'celltype.l2'])

celltype_dict = {k: [] for k in sorted(types_l1)}
for i in range(len(celltype_df)):
    level_1 = celltype_df.iloc[i, 0]
    level_2 = celltype_df.iloc[i, 1]
    if level_2 not in celltype_dict[level_1]:
        celltype_dict[level_1].append(level_2)

celltype_dict

{'B': ['B intermediate', 'B memory', 'B naive', 'Plasmablast'],
 'CD4 T': ['CD4 CTL',
  'CD4 Naive',
  'CD4 Proliferating',
  'CD4 TCM',
  'CD4 TEM',
  'Treg'],
 'CD8 T': ['CD8 Naive', 'CD8 Proliferating', 'CD8 TCM', 'CD8 TEM'],
 'DC': ['ASDC', 'cDC1', 'cDC2', 'pDC'],
 'Mono': ['CD14 Mono', 'CD16 Mono'],
 'NK': ['NK', 'NK Proliferating', 'NK_CD56bright'],
 'other': ['Doublet', 'Eryth', 'HSPC', 'ILC', 'Platelet'],
 'other T': ['MAIT', 'dnT', 'gdT']}

In [16]:
# create binary labels for B naive
# TODO：create labels without creating adata_raw.obs['is B naive']
# adata_raw.obs['is B naive'] = ['B_naive' if x ==
#                                'B naive' else "False" for x in adata_raw.obs['celltype.l2']]
# labels = adata_raw.obs['is B naive'].tolist()
labels = [1 if x == 'B naive' else 0 for x in adata_raw.obs['celltype.l2']]

# create index for B naive
b_naive_indices = [idx for idx, cell_type in enumerate(
    adata_raw.obs['celltype.l2']) if cell_type == 'B naive']


In [20]:
# standardize expression data
adata_std = copy.deepcopy(adata_raw)
# Total-count normalize the data matrix X to 10,000 reads per cell
sc.pp.normalize_total(adata_std, target_sum=1e4)
# Log
sc.pp.log1p(adata_std)


In [29]:
pvl = get_prevalence(adata_raw.X, b_naive_indices)


In [30]:
# pvl = get_prevalence(adata_raw.X, np.arange(adata_raw.shape[0]))
pvl = get_prevalence(adata_raw.X, b_naive_indices)
res = ADlasso2(lmbd=1e-5, echo=True, device='cpu')

st = time.time()
res.fit(adata_std.X, labels, pvl)  # .fit(X, y, prevalence)
et = time.time()

print('median of selected prevalence :', np.median(
    [pvl[i] for i, w in enumerate(res.feature_set) if w != 0]))
print('total selected feature :', np.sum(res.feature_set))
print("Total cost：%f sec" % (et - st))

res.writeList('./Bnaive_selectedList.txt', adata_std.X.columns)


median of selected prevalence : 0.06309924850997668
total selected feature : 13059
Total cost：3420.610412 sec


AttributeError: columns not found

In [39]:
print('median of selected prevalence :', np.median(
    [pvl[i] for i, w in enumerate(res.feature_set) if w != 0]))
print('total selected feature :', np.sum(res.feature_set))
print("Total cost：%f sec" % (et - st))

median of selected prevalence : 0.06309924850997668
total selected feature : 13059
Total cost：3420.610412 sec


In [45]:
res.writeList('./Bnaive_selectedList.txt', adata_raw.var_names)


TypeError: can only concatenate str (not "numpy.int64") to str

In [42]:
adata_raw.var_names[res.feature_set != 0]

Index(['ENSG00000237491', 'ENSG00000225880', 'ENSG00000230368',
       'ENSG00000188976', 'ENSG00000187961', 'ENSG00000188290',
       'ENSG00000187608', 'ENSG00000188157', 'ENSG00000131591',
       'ENSG00000186891',
       ...
       'ENSG00000223653', 'ENSG00000115008', 'ENSG00000117525',
       'ENSG00000237541', 'ENSG00000255823', 'ENSG00000176979',
       'ENSG00000221933', 'ENSG00000229807', 'ENSG00000268400',
       'ENSG00000276345'],
      dtype='object', name='gene_ids', length=13059)

In [46]:
res.feature_sort

array([ 8560, 11469,  6476, ..., 13023, 13069, 20567])

In [49]:

# replace "selected_features.txt" with the desired file name
filename = "bnaive_feature_set.txt"

# extract the selected feature names
selected_features = adata_raw.var_names[res.feature_set != 0]

# save the selected feature names to a text file
np.savetxt(filename, selected_features, fmt="%s")


# replace "feature_sort.txt" with the desired file name
filename = "bnaive_feature_sort.txt"

# save the feature sort to a text file
np.savetxt(filename, res.feature_sort, fmt="%s")

In [ ]:
# Lambda tuning
# auto_scale(X_input, X_raw, y, step=50)
log_lmbd_range = auto_scale(adata_std.X, adata_raw.X, labels, step=50)

lmbd_range = np.exp(log_lmbd_range)
print(lmbd_range)
